In [1]:
%load_ext autoreload

In [ ]:
%autoreload 2

In [2]:
import bittensor
import os
import torch
import torch.multiprocessing as mp 
import time
from loguru import logger
from termcolor import colored
import nest_asyncio 
nest_asyncio.apply()

# Set logging levels.
bittensor.STDOUT_LOGGING_LEVEL = 'ERROR'
bittensor.FILE_LOGGING_LEVEL = 'DEBUG'

# **Init Bittensor** <a class="anchor" id="Init-Bittensor"></a>

## **Wallet** <a class="anchor" id="Wallet"></a>

In [3]:
# WALLET: Holds keys to run your miner and control funds.

# *****
# IMPORTANT: Store the mnemonic for **both** your hot and coldkey 
# you will need these to recover your keys if you forget your password or lose access to this machine.
# ******

# Fill in below to name your wallet and keys.
YOUR_WALLET_NAME = 'colab'
YOUR_HOTKEY_NAME = 'colab_hot'

# Fill in below if your need to regenerate your keys.
use_mnemonic = False # Set to true for key regeneration.
coldkey_mnemonic = "<to be filled>".split(' ')
hotkey_mnemonic = "<to be filled>".split(' ')

# Create the wallet object.
wallet = bittensor.Wallet(
    path = "~/.bittensor/wallets/",
    name = YOUR_WALLET_NAME,
    hotkey = YOUR_HOTKEY_NAME
)

# Optionally regens/creates your wallet keys.
if not wallet.has_coldkeypub:
    if use_mnemonic:
        wallet.regenerate_coldkey(mnemonic = coldkey_mnemonic, use_password = True)
    else:
        wallet.create_new_coldkey(n_words = 12, use_password = True )
if not wallet.has_hotkey:
    if use_mnemonic:
        wallet.regenerate_hotkey(mnemonic = hotkey_mnemonic)
    else:
        wallet.create_new_hotkey(n_words = 12)

# Assert before continuing
assert wallet.has_hotkey
assert wallet.has_coldkeypub

Loaded coldkey.pub: 0x3c9cd1679888e5660b0c8e4b8a17a1719c0cb7f05b5c624a856b421b52290515
Loaded hotkey: 0x80cacfbdf7b155b39de22680a7cb14c61a8f95df702c92f5f142d25cca37c545


## **Subtensor**


In [4]:
if 'subtensor' in locals():
    del subtensor
subtensor = bittensor.Subtensor(
    subtensor_network = 'kusanagi'
)
print (bittensor.Config.toString(subtensor.config))
subtensor.connect()


subtensor:
  chain_endpoint: null
  network: kusanagi
wallet:
  hotkey: default
  name: default
  path: ~/.bittensor/wallets/

Successfully connected to kusanagi endpoint: 159.65.236.189:9944


True

## **Metagraph** <a class="anchor" id="Metagraph"></a> 


In [5]:
if 'metagraph' in locals():
    del metagraph
metagraph = bittensor.Metagraph(
    subtensor = subtensor,
)

In [6]:
metagraph.sync()

...........................................................................................................................................................................................................................................................................................................................................................................................................................................................................



In [7]:
chain_block = metagraph.block()
uids_on_chain = metagraph.uids()
n_neurons = torch.max( metagraph.uids() )
n_online = torch.numel(torch.where( metagraph.block() - metagraph.lastemit() < 1000 )[0])
print ('The chain is at block: {}\n'.format(metagraph.block()))
print ('There are {} subscribed miner neurons \n'.format(n_neurons))
print ('These are their uids: \n{}\n'.format(metagraph.uids()))
print ('{} have set weights in the last 1000 blocks\n'.format(n_online))
print ('\u03C4{} is staked'.format(torch.sum(metagraph.S())))

The chain is at block: 1145634

There are 458 subscribed miner neurons 

These are their uids: 
tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
         42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
         56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
         70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
         84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
         98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
        112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
        126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
        140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
        154, 155, 156, 157, 158, 159, 160, 161

## **Dendrite**

In [8]:
if 'dendrite' in locals():
    del dendrite
dendrite = bittensor.Dendrite(
    wallet = wallet,
    dendrite_multiprocess = False,
    receptor_do_backoff = False,
    receptor_forward_timeout = 50,
    receptor_backward_timeout = 50,
)
print (bittensor.Config.toString(dendrite.config))



dendrite:
  multiprocess: false
receptor:
  backward_timeout: 50
  do_backoff: false
  forward_timeout: 50
  max_backoff: 100
  pass_gradients: true
wallet:
  hotkey: colab_hot
  name: colab
  path: ~/.bittensor/wallets/



# **Nucleus**  <a class="anchor" id="Nucleus"></a>
**a.k.a your unique machine learning model**

## **GPT2-Nucleus** <a class="anchor" id="GPT2-Nucleus"></a>

In [30]:
import torch.nn as nn
import torch.nn.functional as F
from types import SimpleNamespace
from transformers import GPT2Config, GPT2Model

class GPT2Nucleus(torch.nn.Module):
    """ A simple-as-it-gets bittensor nucleus using a GPT2 kernel
    """
    def __init__(self):
        super().__init__()
        huggingface_config = GPT2Config( vocab_size=bittensor.__vocab_size__, n_embd=bittensor.__network_dim__, n_layer=2, n_head=1, n_inner=8 )
        self.transformer = GPT2Model(huggingface_config)
        self.hidden_layer = nn.Linear( bittensor.__network_dim__, bittensor.__network_dim__ )
        self.target_layer = nn.Linear( bittensor.__network_dim__, bittensor.__vocab_size__, bias=False )
        self.loss_fct = nn.CrossEntropyLoss()
        
        # The scores you learn for other neurons in the network.
        self.row_weights = torch.ones([1], requires_grad=True)
        
        # Bittensor components that need to be set before you call remote_forward.
        self.metagraph = None
        self.dendrite = None
    
    
    def local_forward(self, inputs: torch.LongTensor) -> SimpleNamespace:
        """ Applies a forward pass to the model **without** queries to the network.
        """
        # To be filled.
        output = SimpleNamespace()

        # Apply our GPT transformer model.
        # local_context.shape = [ batch_size, sequence_len, network_dim ]
        output.local_context = self.transformer(input_ids=inputs, return_dict=True).last_hidden_state

        # Apply our dense layer and project it onto our hidden layer.
        # local_hidden.shape = [ batch_size, sequence_len, network_dim ]
        output.local_hidden = self.hidden_layer( output.local_context )

        # Project to our target dimension.
        # local_targets.shape = [ batch_size, sequence_len, vocab_size ]
        output.local_targets = self.target_layer( output.local_hidden )

        # Compute LM-loss 
        shift_targets = output.local_targets[..., :-1, :].contiguous()
        shift_inputs = inputs[..., 1:].contiguous()
        output.local_loss = self.loss_fct(shift_targets.view(-1, shift_targets.size(-1)), shift_inputs.view(-1))

        return output

    def remote_forward( self, inputs: torch.LongTensor, n_to_query:int = 10) -> SimpleNamespace:
        """ Applies a forward pass to the model **with** queries to the network.
        """
        # Sanity checks.
        assert self.metagraph != None, 'you must assign model.metagraph, before you run a remote call.'
        assert self.dendrite != None, 'you must assign model.dendrite, before you run a remote call.'
        
        # Run the local part of the model.
        output = self.local_forward( inputs )

        # Make queries to the network.
        output = self.filter_and_make_queries( output, inputs, n_to_query )
        
        # Compute the distillation loss between the local and remote context (produced by the network query.)
        output.distillation_loss = F.mse_loss( output.local_context, output.remote_context.detach() )

        # Apply the hidden dense layer to the context.
        # remote_hidden.shape = [ batch_size, sequence_len, network_dim ]
        output.remote_hidden = self.hidden_layer( output.remote_context )

        # Project onto our target dimension.
        # remote_hidden.shape = [ batch_size, sequence_len, vocab_size ]
        output.remote_targets = self.target_layer( output.remote_hidden )

        # Compute our loss against the remote context.
        shift_targets = output.remote_targets[..., :-1, :].contiguous()
        shift_inputs = inputs[..., 1:].contiguous()
        output.remote_loss = self.loss_fct(shift_targets.view(-1, shift_targets.size(-1)), shift_inputs.view(-1))

        return output
    
    def filter_and_make_queries(self, output: SimpleNamespace, inputs:torch.LongTensor, n_to_query:int = 10) -> SimpleNamespace:
        """ Filters peers based on activity and makes RPC queries.
        """
        
        # Pad the row weights to the network dimension.
        self.row_weights = torch.nn.functional.pad(
            self.row_weights, 
            pad = [0, self.metagraph.n() - self.row_weights.numel() ],
            value = torch.mean(self.row_weights).item() # New values at the mean.
        ).clone().detach().requires_grad_(True)
       
        # Get all neuron uids.
        # all_uids = [n]
        all_uids = metagraph.uids() 
        
        # Filter uids based on last emit.
        # filtered_uids = [ m <= n]
        filtered_uids = all_uids[ torch.where( metagraph.block() - metagraph.lastemit() < 1000 ) ] 

        # Get corresponding weights.
        # filtered_weights = [ m ]
        filtered_weights = self.row_weights[ filtered_uids ]

        # Get topk weights for filtered uids
        # query_indices = [ n_to_query <= m ]
        gamma = 0.9
        output.query_weights, indices = torch.topk(
            filtered_weights + torch.rand_like(filtered_weights) * gamma, 
            min(n_to_query, torch.numel(filtered_weights))
        )
        
        # Duplicate inputs for each request.
        # inputs_to_send = n_to_query * [ batch_size, sequence_length ]
        output.query_uids = filtered_uids[ indices.tolist() ]
        inputs_to_send = [ inputs for _ in output.query_uids.tolist() ]
        neurons_to_query = [ metagraph.neurons()[ i ] for i in output.query_uids.tolist() ]
        
        # Make network calls. 
        # responses = n_to_query * [batch_size, sequence_length, network_dimension]
        output.codes, responses = self.dendrite.forward_text( 
            neurons = neurons_to_query, 
            inputs = inputs_to_send
        )
        
        # Weight-join responses.
        # remote_context = [batch_size, sequence_length, network dimension]
        stacked_responses = torch.stack( responses, dim=2 )
        output.remote_context = torch.matmul( torch.transpose( stacked_responses, dim0=2, dim1=3), output.query_weights)

        return output

In [31]:
# Your nucleus
model = GPT2Nucleus()
model.metagraph = metagraph
model.dendrite = dendrite

## **Test-Nucleus** <a class="anchor" id="Test-Nucleus"></a>

In [32]:
# Test remote forward call.
inputs = torch.tensor([ 
    bittensor.__tokenizer__()('the cat', max_length=10, truncation=True)['input_ids'],  # Text sequence 1
])
output = model.remote_forward( inputs, n_to_query = 50 )
loss = output.local_loss + output.remote_loss + output.distillation_loss
loss.backward() #
print ('The joined remote context from the network is: \n{}\nwith shape: {}\n'.format(output.remote_context, output.remote_context.shape))
print ('The distillation loss between your local and remote context is  {}\n'.format(output.distillation_loss))
print ('The loss with respect to your local context and the targets is {}\n'.format(output.local_loss))
print ('The loss with respect to your remote context and the targets is {}\n'.format(output.remote_loss))
print ('You queried {} remote neurons with uids\n {}\n\nand response codes\n {}\n'.format(torch.numel(output.codes), output.query_uids, output.codes.tolist()))
print ('Gradients w.r.t your row weights: \n', [float('{:0.3f}'.format(model.row_weights.grad[idx].item())) for idx in output.query_uids.tolist()])


2021-04-08 07:25:58.309 | DEBUG    | bittensor.dendrite:forward:151 - -> Forward n_inputs = 50
2021-04-08 07:25:58.311 | DEBUG    | bittensor.dendrite:get_receptor_for_neuron:424 - Create receptor for neuron: version: "1.0.5"
public_key: "0x584fcd177b9f51b84527d462710e590b1c3dd2226c17eb4d40b006a5f1b62a01"
address: "104.248.226.220"
port: 8091
ip_type: 4
uid: 228

2021-04-08 07:25:58.312 | DEBUG    | bittensor.receptor:__init__:142 - New receptor: 104.248.226.220:8091<grpc._channel.Channel object at 0x173696f60><bittensor.bittensor_pb2_grpc.BittensorStub object at 0x1736966d8>
2021-04-08 07:25:58.312 | DEBUG    | bittensor.dendrite:get_receptor_for_neuron:424 - Create receptor for neuron: version: "1.0.5"
public_key: "0x960c483e17d8da4ef92eef1a057bf9de99d34f0449befffd91d89406a462960c"
address: "161.35.97.185"
port: 8091
ip_type: 4
uid: 74

2021-04-08 07:25:58.313 | DEBUG    | bittensor.receptor:__init__:142 - New receptor: 161.35.97.185:8091<grpc._channel.Channel object at 0x173696080><

# **Training** <a class="anchor" id="Training"></a>

## **Training-Loop** <a class="anchor" id="Training-Loop"></a>

In [34]:
from torch.nn.utils import clip_grad_norm_
import torch.nn.functional as F
from loguru import logger
import random
from datasets import load_dataset
import time
import os

# ---- Dataset ---- 
dataset = load_dataset('ag_news')['train']
def nextbatch(data, batch_size, tokenizer):
    """ Returns a random batch of sentences from text dataset.
    """
    batch_text = []
    for _ in range(batch_size):
        batch_text.append(data[random.randint(0, len(data))]['text'])
    batch_inputs = tokenizer(batch_text, return_tensors='pt', max_length=10, padding=True, truncation=True)['input_ids']
    return batch_inputs

# --- Training Logger ----
if 'training_logger' not in locals():
    training_log_dir = os.path.expanduser('~/logs/training.log')
    logger.add(training_log_dir, filter=lambda record: record["extra"].get("name") == "training", enqueue=True, backtrace=True, diagnose=True, rotation="500 MB")
    training_logger = logger.bind(name="training")
def show_training_logs(length: int = 25):
    ! tail -n $length $training_log_dir

# ---- Tokenizer ----
# For encoding text inputs.
tokenizer = bittensor.__tokenizer__()

# ---- Optimizer ----
# For applying gradient steps to the local model.
optimizer = torch.optim.SGD( model.parameters(), lr = 0.1, momentum = 0.99 )

# ---- Training Loop -----
def train( 
        stop_training: mp.Event,
    ):
    # ---- Loop until event is set ----
    training_step = 0
    batch_size = 1
    logger.bind(training=True).info('Loop starting... ')
    while not stop_training.is_set():
        try:
            optimizer.zero_grad() # Zeros out gradients for next accummulation

            # ---- Forward pass ----
            inputs = nextbatch( dataset, batch_size, tokenizer )
            output = model.remote_forward( inputs, n_to_query=10 )

            # ---- Backward pass ----
            loss = output.local_loss + output.remote_loss + output.distillation_loss
            loss.backward() # Accumulates gradients on the model.
            clip_grad_norm_(model.parameters(), 0.8) # clip model gradients
            optimizer.step() # Applies accumulated gradients.

            # ---- Step logs ----
            training_logger.info('->\nuids:{}\ncodes:{}\nweights:{}\ngrads:{}', 
                  output.query_uids, 
                  output.codes.tolist(), 
                  [float('{:0.3f}'.format(x)) for x in output.query_weights.tolist()],
                  [float('{:0.3f}'.format(model.row_weights.grad[idx].item())) for idx in output.query_uids.tolist()])      
            training_logger.info('gs:{} loss(local):{} loss(remote):{} loss(distill):{} dendrite:{}',
                  colored('{}'.format( training_step ), 'red'),
                  colored('{:.4f}'.format(output.local_loss.item()), 'green'),
                  colored('{:.4f}'.format(output.remote_loss.item()), 'blue'),
                  colored('{:.4f}'.format(output.distillation_loss.item()), 'red'),
                  dendrite)

            # --- Train and normalize row weights ---
            model.row_weights = torch.clamp(model.row_weights - 0.001 * model.row_weights.grad, 0, 1)
            model.row_weights = F.normalize( model.row_weights, p = 1, dim = 0 ).clone().detach().requires_grad_(True)

            training_step += 1
        except Exception as e:
            training_logger.exception("Training iteration exception.")
    logger.bind(training=True).complete()

Using custom data configuration default
Reusing dataset ag_news (/Users/const/.cache/huggingface/datasets/ag_news/default/0.0.0/fb5c5e74a110037311ef5e904583ce9f8b9fbc1354290f97b4929f01b3f48b1a)


## **Training Thread Runners** <a class="anchor" id="Training-Thread-Runners"></a>

In [35]:
import threading
import torch.multiprocessing as mp 
import sys

join_timeout = 10

if 'quit_training' in locals():
    quit_training.set()
if 'training_thread' in locals() and training_thread.is_alive():
    training_thread.join( timeout = join_timeout )

quit_training = mp.Event()
training_thread = threading.Thread( target = train, args = (quit_training,),  name = 'training', daemon=True)

def stop_training():
    global quit_training
    global training_thread
    quit_training.set()
    if not training_thread.is_alive():
        return
    training_logger.info("Joining...")
    training_thread.join( timeout = join_timeout )
    if not training_thread.is_alive():
        print ('Joined training thread',)
        training_logger.info('Joined.')
    else:
        print ('Failed to join training thread')

def start_training():
    global quit_training
    global training_thread
    stop_training()
    quit_training = mp.Event()
    training_thread = threading.Thread( target = train, args = (quit_training,), name = 'training', daemon=True)
    training_thread.start()
    training_logger.info("Started training.")
    print('new training thread:', training_thread)

In [36]:
start_training()

new training thread: <Thread(training, started daemon 123145804439552)>
2021-04-08 07:26:57.950 | DEBUG    | bittensor.dendrite:forward:151 - -> Forward n_inputs = 10
2021-04-08 07:26:57.951 | DEBUG    | bittensor.dendrite:get_receptor_for_neuron:424 - Create receptor for neuron: version: "1.0.5"
public_key: "0x5ca06f5875cde8f08a57cf3ac01eb1ff414c5c4aad4115be8a48a0b90badb249"
address: "142.93.12.194"
port: 8091
ip_type: 4
uid: 182

2021-04-08 07:26:57.952 | DEBUG    | bittensor.receptor:__init__:142 - New receptor: 142.93.12.194:8091<grpc._channel.Channel object at 0x173696198><bittensor.bittensor_pb2_grpc.BittensorStub object at 0x16fe1e908>
2021-04-08 07:26:57.952 | DEBUG    | bittensor.receptor:_call_forward:357 - -> Forward rpc to: 142.93.12.194:8091
2021-04-08 07:26:57.954 | DEBUG    | bittensor.receptor:_call_forward:357 - -> Forward rpc to: 104.248.226.93:8091
2021-04-08 07:26:57.954 | DEBUG    | bittensor.receptor:_call_forward:357 - -> Forward rpc to: 204.48.22.224:8091
2021-0

In [45]:
print (training_thread.is_alive())

True


In [39]:
stop_training()

Failed to join training thread


In [40]:
show_training_logs(50)

grads:[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
2021-04-07 18:37:01.662 | INFO     | __main__:train:69 - gs:2 loss(local):10.3682 loss(remote):10.8260 loss(distill):0.9996 dendrite:(0.002q/s|⤊ 380.8/⤋ 0.0kB/s)
2021-04-07 18:37:05.602 | INFO     | __main__:stop_training:21 - Joining...
2021-04-07 18:37:14.743 | INFO     | __main__:train:63 - ->
uids:tensor([227, 379, 198, 195, 185,  62, 218, 337, 332,  12, 183,  13, 229,  71,
        385, 186, 175, 216, 184,  57,  55,  65,  64,  14, 174, 207, 219,  48,
        203, 194, 380, 214,  82, 384, 179, 398, 371,   2, 188, 189, 212, 190,
        176, 193, 368, 197, 202, 220, 397, 330])
codes:[1, 1, 1, 1, 1, 1, 1, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


# **Serving** <a class="anchor" id="Serving"></a>

In [42]:
if 'axon' in locals():
    del axon
axon = bittensor.Axon(
    axon_local_ip = '127.0.0.1',
    axon_local_port = 8082,
    axon_external_port = 8082,
    axon_external_ip = bittensor.utils.networking.get_external_ip(),
)
print (bittensor.Config.toString(axon.config))


axon:
  backward_processing_timeout: 5
  external_ip: 190.237.10.119
  external_port: 8082
  forward_processing_timeout: 5
  local_ip: 127.0.0.1
  local_port: 8082
  max_workers: 10
  use_upnpc: false
wallet:
  hotkey: default
  name: default
  path: ~/.bittensor/wallets/



In [43]:
# Start the axon serving endpoint.
axon.start()

2021-04-08 07:28:29.410 | INFO     | bittensor.axon:start:639 - Starting Axon thread.
2021-04-08 07:28:29.411 | INFO     | bittensor.axon:_serve:619 - Axon is serving on: 127.0.0.1:8082
2021-04-08 07:28:35.242 | DEBUG    | bittensor.dendrite:forward:157 - <- Forward messages: [(0, 'success'), (3, 'grpc unavailable'), (0, 'success'), (3, 'grpc unavailable'), (0, 'success'), (3, 'grpc unavailable'), (0, 'success'), (0, 'success'), (0, 'success'), (0, 'success')]


In [ ]:
axon.stop()

In [49]:
! tail -n 10 '~/.bittensor/logs.log'

2021-04-08 07:28:35.242 | DEBUG    | bittensor.dendrite:forward:157 - <- Forward messages: [(0, 'success'), (3, 'grpc unavailable'), (0, 'success'), (3, 'grpc unavailable'), (0, 'success'), (3, 'grpc unavailable'), (0, 'success'), (0, 'success'), (0, 'success'), (0, 'success')]
2021-04-08 07:28:35.280 | DEBUG    | bittensor.dendrite:backward:184 - -> Backward n_inputs = 10
2021-04-08 07:28:35.283 | DEBUG    | bittensor.receptor:_call_backward:479 - -> Backward rpc to: 204.48.31.202:8091
2021-04-08 07:28:35.284 | DEBUG    | bittensor.receptor:_call_backward:479 - -> Backward rpc to: 142.93.251.167:8091
2021-04-08 07:28:35.286 | DEBUG    | bittensor.receptor:_call_backward:479 - -> Backward rpc to: 204.48.28.226:8091
2021-04-08 07:28:35.288 | DEBUG    | bittensor.receptor:_call_backward:479 - -> Backward rpc to: 142.93.13.96:8091
2021-04-08 07:28:35.289 | DEBUG    | bittensor.receptor:_call_backward:479 - -> Backward rpc to: 206.81.2.135:8091
2021-04-08 07:28:35.290 | DEBUG    | bittenso

In [50]:
import grpc 
channel = grpc.insecure_channel(
            '127.0.0.1:8082',
            options=[('grpc.max_send_message_length', -1),
                     ('grpc.max_receive_message_length', -1)])
stub = bittensor.grpc.BittensorStub( channel )


inputs_raw = torch.tensor( [ [ 1 ] ], dtype=torch.int64)
serializer = bittensor.serialization.get_serializer( serialzer_type = bittensor.proto.Serializer.MSGPACK )
inputs_serialized = serializer.serialize(inputs_raw, modality = bittensor.proto.Modality.TEXT, from_type = bittensor.proto.TensorType.TORCH)
request = bittensor.proto.TensorMessage(
    version = bittensor.__version__,
    public_key = 'sssss',
    tensors = [inputs_serialized]
)

response = stub.Forward(request)

2021-04-08 07:39:04.941 | DEBUG    | bittensor.axon:Forward:187 - -> Forward request: sssss, size:80
2021-04-08 07:39:04.942 | DEBUG    | bittensor.axon:enqueue_forward_to_nucleus:315 - enqueue_forward_to_nucleus: sssss, inputs_x: tensor([[1]])


[autoreload of bittensor.dendrite failed: Traceback (most recent call last):
  File "/Users/const/.pyenv/versions/3.7.3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/const/.pyenv/versions/3.7.3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/const/.pyenv/versions/3.7.3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/Users/const/.pyenv/versions/3.7.3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 302, in update_class
    if update_generic(old_obj, new_obj): continue
  File "/Users/const/.pyenv/versions/3.7.3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/Users/const/.pyenv/versions/3.7.3/lib/python3.7/site-packages/IPython/extensions/autoreload.

2021-04-08 07:39:09.947 | DEBUG    | bittensor.axon:enqueue_forward_to_nucleus:339 - processing timeout
Loaded hotkey: 0xbc8ca2afe5a3bca80254bc51ab9fd9f88a5891bd3ecb8e10f917170b0043ff69
2021-04-08 07:39:09.953 | DEBUG    | bittensor.axon:Forward:196 - <- Forward response: 0xbc8ca2afe5a3bca80254bc51ab9fd9f88a5891bd3ecb8e10f917170b0043ff69, size:80


## **Serving-Loop** <a class="anchor" id="Serving-Loop"></a>

In [52]:
# ---- Serving logger ----
if 'serving_logger' not in locals():
    serving_log_dir = os.path.expanduser('~/logs/serving.log')
    logger.add(serving_log_dir, colorize=True, filter=lambda record: record["extra"].get("name") == "serving", enqueue=True, backtrace=True, diagnose=True, rotation="500 MB")
    serving_logger = logger.bind(name="serving")

def show_serving_logs(length: int = 25):
    ! tail -n $length $serving_log_dir

# ---- Serving loop -----
def serve ( 
  stop_serving: mp.Event,
):

    # ---- Loop until event is set -----
    serving_step = 0
    serving_logger.info('Serving thread started: ')
    while not stop_serving.is_set():
        try:
            
            # ---- Pull request ----
            serving_logger.info('Axon:{}, waiting for query ... ', axon)
            pong, pubkey, inputs, modality = axon.next_forward_item( timeout = 10.0 )

            # ---- Process request ----
            if None not in [ pong, pubkey, inputs, modality ]:
                serving_logger.info('Recieved Query: from:{}, inputs.shape:{}', pubkey, inputs.shape)
                output = model.local_forward( inputs ).local_hidden
                pong.send( output.detach() )
                serving_logger.info('Sent response: to:{}, output.shape:{}', pubkey, output.shape)

        except Exception as e:
            serving_logger.exception('Error in forward process with error {}', e)



## **Serving Thread Runners** <a class="anchor" id="Serving-Thread-Runners"></a>

In [53]:
import threading
join_timeout = 10

if 'quit_serving' in locals():
    quit_serving.set()
if 'serving_thread' in locals() and serving_thread.is_alive():
    serving_thread.join( timeout = join_timeout )

quit_serving = mp.Event()
serving_thread = threading.Thread( target = serve, args = (quit_serving,),  name = 'serving', daemon=True )

def stop_serving():
    global quit_serving
    global serving_thread
    quit_serving.set()
    if serving_thread.is_alive():
        serving_logger.info("Joining...")
        serving_thread.join( timeout = join_timeout )
    if not serving_thread.is_alive():
        print ('Joined serving thread',)
        serving_logger.info('Joined.')
    else:
        print ('Failed to join serving thread')

def start_serving():
    global quit_serving
    global serving_thread
    stop_serving()
    quit_serving = mp.Event()
    serving_thread = threading.Thread( target = serve, args = (quit_serving,), name = 'serving', daemon=True )
    serving_thread.start()
    serving_logger.info("Started serving.")
    print('new serving thread:', serving_thread)


In [54]:
start_serving()

Joined serving thread
new serving thread: <Thread(serving, started daemon 123145825460224)>


In [55]:
print (serving_thread.is_alive())

True


In [ ]:
stop_serving()

In [63]:
show_serving_logs(20)

  File "/Users/const/.pyenv/versions/3.7.3/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
    │    │     │        └ <memory at 0x173971b88>
    │    │     └ b'\x00\x00\x01>'
    │    └ <function Connection._send at 0x1368b4268>
    └ <multiprocessing.connection.Connection object at 0x1735acac8>
  File "/Users/const/.pyenv/versions/3.7.3/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
        │     │    │        └ b'\x00\x00\x01>\x80\x03ctorch.multiprocessing.reductions\nrebuild_tensor\nq\x00ctorch\nTensor\nq\x01ctorch.multiprocessing.re...
        │     │    └ 233
        │     └ <multiprocessing.connection.Connection object at 0x1735acac8>
        └ <built-in function write>

BrokenPipeError: [Errno 32] Broken pipe
2021-04-08 07:39:25.502 | INFO     | __main__:serve:22 - Axon:(0.000q/s|⤊ 0.0/⤋ 0.0kB/s), waiting for query ... 
2021-04-08 07:39:35.504 | INFO     | __main__:serve:22 - A

In [62]:
endpoint = bittensor.proto.Neuron(
    address = axon.config.axon.local_ip,
    port = axon.config.axon.local_port,
    public_key = wallet.hotkey.public_key
)
start_time = time.time()
codes, responses = dendrite.forward_text( 
    neurons = [ endpoint ],
    inputs = [ torch.tensor([[1]]) ]
)
end_time = time.time()
print(colored('Querying endpoint: {}:{}'.format(axon.config.axon.local_ip, axon.config.axon.local_port), 'blue'))
if codes.item() == bittensor.proto.ReturnCode.Success:
    print(colored('Success', 'green'))
    print(colored('Response shape: {}'.format(responses[0].shape) , 'green'))
    print(colored('Query time: {}'.format(end_time - start_time) , 'green'))
else:
    print(colored('Failure with code: {}'.format(codes.item()), 'red'))
    print(colored('Ensure your axon is started with axon.start()', 'red'))
    print(colored('Ensure your endpoint is accessible from the internet, perhaps behind your router\'s NAT?', 'red'))


2021-04-08 07:39:41.622 | DEBUG    | bittensor.dendrite:forward:151 - -> Forward n_inputs = 1
2021-04-08 07:39:41.624 | DEBUG    | bittensor.dendrite:get_receptor_for_neuron:424 - Create receptor for neuron: public_key: "0x80cacfbdf7b155b39de22680a7cb14c61a8f95df702c92f5f142d25cca37c545"
address: "127.0.0.1"
port: 8082

2021-04-08 07:39:41.625 | DEBUG    | bittensor.receptor:__init__:142 - New receptor: 127.0.0.1:8082<grpc._channel.Channel object at 0x15d29c9b0><bittensor.bittensor_pb2_grpc.BittensorStub object at 0x1717fc8d0>
2021-04-08 07:39:41.627 | DEBUG    | bittensor.receptor:_call_forward:357 - -> Forward rpc to: 127.0.0.1:8082
2021-04-08 07:39:41.628 | DEBUG    | bittensor.axon:Forward:187 - -> Forward request: 0x80cacfbdf7b155b39de22680a7cb14c61a8f95df702c92f5f142d25cca37c545, size:80
2021-04-08 07:39:41.629 | DEBUG    | bittensor.axon:enqueue_forward_to_nucleus:315 - enqueue_forward_to_nucleus: 0x80cacfbdf7b155b39de22680a7cb14c61a8f95df702c92f5f142d25cca37c545, inputs_x: tens

# **Weights**

## **Filtering Weights**

In [ ]:
# Get the trained weights from the chain.
weights_to_emit = torch.nn.functional.pad(
    model.row_weights, 
    pad = [0, metagraph.n() - model.row_weights.numel() ],
    value = torch.mean(model.row_weights).item()
)

model.row_weights.detach()

# Take topk
topk = 30
weights_to_emit, uids = torch.topk(weights_to_emit, topk)

# Normalize to 0,1
weights_to_emit = F.normalize(weights_to_emit, p = 1, dim = 0)
print ("Weights:\n{}\nFor uids\n {}".format(weights_to_emit.tolist(), uids.tolist()))



## **Setting Weights**

In [ ]:
# Sets your incentive weights on the chain.
subtensor.set_weights(
    uids = uids,
    weights = weights_to_emit
)

In [ ]:
your_uid = bittensor.metagraph.uids()[0]
print ('Your weights (encoded as uint32s) on the chain are: \n\n {} \n'.format(bittensor.subtensor.weight_vals_for_uid( your_uid )))

print ('For uids \n {}'.format(bittensor.subtensor.weight_uids_for_uid( your_uid )))

# **Transactions** <a class="anchor" id="Transactions"></a>

## **Unstaking-Funds** <a class="anchor" id="Unstaking-Funds"></a>

In [ ]:
from bittensor.utils.balance import Balance
amount_tao = 0.1
amount = Balance.from_float( amount_tao )
print(colored("Sending Extrinsic: [Unstake: {} Tao from hotkey: {}]".format( amount.tao, wallet.hotkey.public_key) , 'blue'))
print ('waiting for finalization...')
result = subtensor.unstake( amount, wallet.hotkey.public_key, wait_for_finalization = True, timeout = bittensor.__blocktime__ * 5)
if result:
    new_balance = subtensor.get_balance(wallet.coldkeypub)
    new_stake = subtensor.get_stake_for_uid( metagraph.uid_for_pubkey(wallet.hotkey.public_key) )
    print(colored("Unstaked: {} Tao from hotkey: {} to coldkey.pub: {}".format( amount.tao, wallet.hotkey.public_key, wallet.coldkey.public_key ) , 'green'))
    print(colored("Your coldkey has new balance: {} Tao".format( new_balance.tao ) , 'green'))
    print(colored("Your hotkey has new stake: {} Tao".format( new_stake.tao ) , 'green'))
else:
    print(colored("Unstaking transaction failed", 'red'))

## **Staking-Funds** <a class="anchor" id="Staking-Funds"></a>

In [ ]:
from bittensor.utils.balance import Balance 
amount_tao = 0.1
amount = Balance.from_float( amount_tao )
print(colored("Sending Extrinsic: [Stake: {} Tao to hotkey: {}]".format( amount.tao, wallet.hotkey.public_key) , 'blue'))
print ('waiting for finalization...')
result = subtensor.add_stake( amount, wallet.hotkey.public_key, wait_for_finalization = True, timeout = bittensor.__blocktime__ * 5)
if result:
  new_balance = subtensor.get_balance(wallet.coldkeypub)
  new_stake = subtensor.get_stake_for_uid( metagraph.uid_for_pubkey(wallet.hotkey.public_key))
  print(colored("Staked: {} Tao to hotkey: {} from coldkey.pub: {}".format( amount.tao, wallet.hotkey.public_key, wallet.coldkey.public_key ) , 'green'))
  print(colored("Your coldkey has new balance: {} Tao".format( new_balance.tao ) , 'green'))
  print(colored("Your hotkey has new stake: {} Tao".format( new_stake.tao ) , 'green'))

else:
  print(colored("Staking transaction failed", 'red'))

## **Transfering-Funds** <a class="anchor" id="Transfering-Funds"></a>

In [ ]:
amount = 0.01
destination_public_key = wallet.coldkey.public_key
amount = Balance.from_float( amount )
balance = bittensor.subtensor.get_balance( wallet.coldkey.public_key )
if balance < amount:
    print(colored("Not enough balance ({}) to transfer {}".format(balance, amount), 'red'))
    quit()

print(colored("Requesting transfer of {} Tao, from coldkey.pub: {} to dest.pub: {}".format(amount.tao, wallet.coldkey.public_key, destination_public_key), 'blue'))
print("Waiting for finalization...",)
result = bittensor.subtensor.transfer(destination_public_key, amount, wait_for_finalization = True, timeout = bittensor.__blocktime__ * 5)
if result:
    print(colored("Transfer finalized with amount: {} Tao to dest: {} from coldkey.pub: {}".format(amount.tao, destination_public_key, wallet.coldkey.public_key), 'green'))
    new_balance = bittensor.subtensor.get_balance(wallet.coldkeypub)
    destination_balance = bittensor.subtensor.get_balance(destination_public_key)
    print(colored("Your coldkey has new balance: {} Tao".format( new_balance.tao ) , 'green'))
    print(colored("The destination has new balance: {} Tao".format( new_balance.tao ) , 'green'))
else:
    print(colored("Transfer failed", 'red'))